# tf.constant 和 tf.Variable 
是 TensorFlow 中用于创建 张量（tensor）对象 的两种基础方式，它们都可以用来表示数据，但功能和用途有所不同。

### 🧱 1. tf.constant
✅ 定义：\
表示不可变（immutable）的张量，创建后值不能修改。

📌 用法：

In [1]:
import tensorflow as tf

a = tf.constant([1.0, 2.0, 3.0])
print(a)

tf.Tensor([1. 2. 3.], shape=(3,), dtype=float32)


### 🧠 特点：
| 特性              | 说明        |
| --------------- | --------- |
| 不可变             | 创建后数值无法更改 |
| 用于模型参数中不需要更新的部分 | 如超参数、常量项  |
| 可用于图构建或计算图输入    |           |

💡 示例：\
c = tf.constant([[1, 2], [3, 4]]) \
c[0][0] = 100  # ❌ 错误：不能修改 constant 的值


---

### 🧱 2. tf.Variable
✅ 定义：\
表示**可变（mutable）**的张量，通常用于表示模型的可训练参数，如神经网络的权重。

📌 用法：

In [ ]:
import tensorflow as tf
v = tf.Variable([1.0, 2.0, 3.0])
print(v)

### 🧠 特点：
| 特性                                   | 说明                     |
| ------------------------------------ | ---------------------- |
| 可变                                   | 值可以被更新                 |
| 常用于机器学习模型的权重                         | 用于 `GradientTape` 自动微分 |
| 支持 `.assign()` 和 `.assign_add()` 等操作 |                        |

💡 示例：\
v = tf.Variable([1, 2, 3]) \
v.assign([4, 5, 6])          # 替换值  \
v.assign_add([1, 1, 1])      # 加法更新  
### ✅ 对比总结表：
| 特征                     | `tf.constant` | `tf.Variable` |
| ---------------------- | ------------- | ------------- |
| 是否可变                   | ❌ 不可变         | ✅ 可变          |
| 是否可训练                  | ❌ 否           | ✅ 是           |
| 是否用于模型参数               | 🚫 一般不用       | ✅ 常用于权重、偏置    |
| 是否支持 `.assign()` 等更新方法 | ❌ 否           | ✅ 是           |
| 自动微分支持                 | ✅ 支持          | ✅ 支持          |

### 📘 举个典型场景：

In [ ]:
# 常量：学习率或标签
learning_rate = tf.constant(0.01)

# 可训练变量：模型权重
W = tf.Variable(tf.random.normal([3, 2]), name='weight')
b = tf.Variable(tf.zeros([2]), name='bias')

# .assign()、.assign_add() 和相关操作方法。
## 🔧 一、.assign() —— 直接赋新值
✅ 功能：将变量的值替换为一个新的值

import tensorflow as tf

v = tf.Variable([1.0, 2.0, 3.0])\
v.assign([4.0, 5.0, 6.0])\
print(v.numpy())  # 输出: [4. 5. 6.]   ---> 查看张量的值使用：tensor.numpy() \
📌 要求新值的形状和原始变量一致，否则报错。

---
## ➕ 二、.assign_add() —— 原值累加
✅ 功能：将变量的值增加一个值（原地加法）

v = tf.Variable([1.0, 2.0, 3.0])\
v.assign_add([10.0, 10.0, 10.0])\
print(v.numpy())  # 输出: [11. 12. 13.]

---
## ➖ 三、.assign_sub() —— 原值累减
✅ 功能：将变量的值减去一个值（原地减法）\

v = tf.Variable([10.0, 10.0, 10.0])\
v.assign_sub([1.0, 2.0, 3.0])\
print(v.numpy())  # 输出: [9. 8. 7.]

---
## 🚨 注意事项
| 操作名             | 含义    | 是否原地操作 | 支持梯度传播 |
| --------------- | ----- | ------ | ------ |
| `.assign()`     | 赋新值   | ✅ 是    | ✅ 支持   |
| `.assign_add()` | 原值加新值 | ✅ 是    | ✅ 支持   |
| `.assign_sub()` | 原值减新值 | ✅ 是    | ✅ 支持   |

-  这些操作必须用于 tf.Variable 类型。
-  如果你对一个 tf.constant 使用 .assign() 会报错。
-  支持在 tf.GradientTape() 环境中使用，依然可正常反向传播。

### 💡 在训练中常见的用法

In [ ]:
# 模拟梯度下降过程
W = tf.Variable(5.0)

with tf.GradientTape() as tape:
    loss = (W - 3.0) ** 2  # 假设损失函数

grad = tape.gradient(loss, W)
W.assign_sub(0.1 * grad)  # 梯度下降一步

print(W.numpy())  # 权重被更新


---

# 📖 tf.GradientTape() 讲解
## 1. 基本概念

- 作用：记录张量的运算过程，从而能够自动计算梯度。

- 用途：常用于神经网络训练，帮我们算出 损失函数对模型参数的梯度。

- 换句话说：\
它相当于“录制机 🎥”，会把你在它上下文中执行的运算步骤都记下来，然后可以“反向播放”，帮你求梯度。

### 👉什么时候使用？：
需要`自定义训练循环`时	使用 tf.GradientTape

## 2. 基本用法

In [ ]:
import tensorflow as tf

# 定义变量
x = tf.Variable(3.0)

# 在 GradientTape 的上下文内进行要记录的操作
with tf.GradientTape() as tape:
    # tape.watch(x)  对于由 tf.constant 定义的变量，tape.watch() 可确保它被追踪
    # tf.Variable 定义的变量 无需手动调用 tape.watch(variable)
    y = x**2  # y = x^2   前向传播计算 y

# 计算 y 关于 x 的梯度 .求 dy/dx
dy_dx = tape.gradient(y, x)
print(dy_dx.numpy())  # 输出： tf.Tensor(6.0, shape=(), dtype=float32)
# 因为 dy/dx = 2x，当 x=3.0 时，结果为 6.0

👉 这里的原理是：
tape 记录了 y = x^2 的计算
tape.gradient(y, x) 自动帮我们算了导数 dy/dx = 2x = 6

## 3. 常见参数
`persistent`=True

- `默认`情况下，`GradientTape` 只能计算`一次`梯度，算完就释放资源。

- 如果想多次计算，需要设置 `persistent` = `True`。切记用完后手动删除磁带：del tape -->  # 重要！手动释放资源

In [ ]:
x = tf.Variable(3.0)

with tf.GradientTape(persistent=True) as tape:
    y = x**2
    z = y * 2

dy_dx = tape.gradient(y, x)  # 6.0
dz_dx = tape.gradient(z, x)  # 12.0
del tape # 重要！手动释放资源

`watch()`

- 变量（`tf.Variable`）会被自动监视（tape 自动追踪它的梯度）。

- 普通张量（`tf.constant`）不会自动追踪，需要手动 tape.watch()。

In [ ]:
x = tf.constant(3.0)

with tf.GradientTape() as tape:
    tape.watch(x)  # 手动监视
    y = x**2

dy_dx = tape.gradient(y, x)
print(dy_dx.numpy())  # 6.0


## 4. 在训练中的应用
通常我们会在训练循环里用 `GradientTape` 来求梯度，再用优化器更新参数。

In [ ]:
# 简单的线性回归：y = w*x + b
w = tf.Variable(2.0)
b = tf.Variable(1.0)

x = tf.constant([1.0, 2.0, 3.0])
y_true = tf.constant([3.0, 5.0, 7.0])  # 真实值

optimizer = tf.keras.optimizers.SGD(learning_rate=0.1)

for step in range(100):
    with tf.GradientTape() as tape:
        y_pred = w * x + b
        loss = tf.reduce_mean(tf.square(y_true - y_pred))  # MSE
    
    # 计算梯度
    grads = tape.gradient(loss, [w, b])
    # 应用梯度
    optimizer.apply_gradients(zip(grads, [w, b]))

print(w.numpy(), b.numpy())  # 接近真实值 w=2, b=1

👉 训练流程：

1.前向传播：计算预测值 & 损失

2.反向传播：用 tape.gradient 求梯度

3.参数更新：优化器根据梯度更新 w 和 b

## 5. 多阶导数（高阶导数）

tf.GradientTape 还能嵌套使用，计算二阶导数。

In [ ]:
x = tf.Variable(3.0)

with tf.GradientTape() as t1:
    with tf.GradientTape() as t0:
        y = x**3  # y = x^3
    dy_dx = t0.gradient(y, x)  # dy/dx = 3x^2 = 27

d2y_dx2 = t1.gradient(dy_dx, x)  # d²y/dx² = 6x = 18
print(dy_dx.numpy(), d2y_dx2.numpy())


## 6.现代训练循环的模板：
在实际项目中，我们通常会使用 Keras Model 和 Optimizer，让代码更简洁。

In [ ]:
# 假设 model 是一个 tf.keras.Model, optimizer 是 tf.keras.optimizers

for epoch in range(epochs):
    for batch_x, batch_y in train_dataset: # 使用 tf.data.Dataset
        with tf.GradientTape() as tape:
            predictions = model(batch_x, training=True) # 前向传播
            loss = loss_function(batch_y, predictions)  # 计算损失

        # 计算梯度
        gradients = tape.gradient(loss, model.trainable_variables)
        # 使用优化器更新权重（代替手动 W.assign_sub）
        optimizer.apply_gradients(zip(gradients, model.trainable_variables))

📊 总结速查表
| 功能   | 方法                                 |
| ---- | ---------------------------------- |
| 开始录制 | `with tf.GradientTape() as tape:`  |
| 计算梯度 | `tape.gradient(loss, var_list)`    |
| 监视张量 | `tape.watch(tensor)`               |
| 多次使用 | `tf.GradientTape(persistent=True)` |
| 高阶导数 | 嵌套 `GradientTape`                  |
